# Arbitrage Algorithm for DEX Pools

- This algorithm is Not Optimized, but marginally optimized.

For generalization of https://ajlab402.blogspot.com/2022/03/osmosis-zone.html experiment.  
Here is the problem settings:  

1. Tokens are traded in CEX and DEX.  
2. We want to make money from the swap ratio difference between pools (or CEX).  
3. We have to finish our trade with no market exposure.
4. We have price data of CEX and token reserves of DEX.
5. Swap fees for all trades are given as 0.1%.
6. CEX ask & bid prices are same, and amount is large enough.

#### Let's get started!

In [1]:
import pandas as pd
import math

In [1]:
# For problem simplication, impose contraints for routing algorithm
# -> Check for length-3 route is quite enough.
# length-1 : CEX_ticker -> CEX_ticker
# length-2 : CEX_ticker -> DEX_ticker -> CEX_ticker
# length-3 : CEX_ticker -> DEX_ticker -> DEX_ticker -> CEX_ticker

In [9]:
reserves = pd.read_csv('reserves.csv', encoding='cp949', index_col=0)

orderbook = pd.read_csv('orderbook.csv', encoding='cp949', index_col=0)

total_asset = list(set(reserves.iloc[:,0].unique().tolist() + reserves.iloc[:,1].unique().tolist()))
total_asset.sort()

threshold = 0.1 # If profit is smaller than threshold, just pass the route.

r = 0.999 # fee
CEX_tickers = total_asset[:8]

In [10]:
pool_network = []
for ticker in total_asset:
    for i in range(reserves.shape[0]):
        if reserves.iloc[i,0] == ticker:
            if reserves.iloc[i,1] in CEX_tickers:
                pool_network.append([ticker, 'CEX', reserves.iloc[i,1], i])
            else:
                pool_network.append([ticker, 'DEX', reserves.iloc[i,1], i])
        elif reserves.iloc[i,1] == ticker:
            if reserves.iloc[i,0] in CEX_tickers:
                pool_network.append([ticker, 'CEX', reserves.iloc[i,0], i])
            else:
                pool_network.append([ticker, 'DEX', reserves.iloc[i,0], i])
                
pool_network = pd.DataFrame(pool_network) # df generation for search
pool_network.columns = ['ticker', 'EX', 'counter', 'reserve_index']

## Routing algorithm

Optimization is simple. Just use differentiation and check first, second order conditions.

In [11]:
# Let's Define search algorithms for various lengths.
def length_1(reserve):
    search_order = CEX_tickers # It is better to randomize or optimize search order.
    cash = 0
    cash_hist = []
    sequence_hist = []
    for ticker in search_order:
        candidates = pool_network[(pool_network['ticker']==ticker) * (pool_network['EX'] == 'CEX')]
        for i in range(candidates.shape[0]):
            pool = reserve.iloc[candidates.iloc[i,3]]
            order_change = pool.iloc[0]==ticker
            real_price = orderbook.loc[candidates.iloc[i,2]]['price']/orderbook.loc[ticker]['price']
            pool_product = pool['amount 1'] * pool['amount 2']
            origin_amount = pool.iloc[3-order_change]
            counter_amount = pool.iloc[2+order_change]
            
            ticker_in = (-origin_amount + math.sqrt(pool_product*real_price))/r
            if ticker_in < 0:
                continue
            counter_out = counter_amount - pool_product/(origin_amount+ticker_in*r)
            
            if counter_out * orderbook.loc[candidates.iloc[i,2]]['price'] - ticker_in * orderbook.loc[ticker]['price'] > threshold:
                cash += counter_out * orderbook.loc[candidates.iloc[i,2]]['price'] - ticker_in * orderbook.loc[ticker]['price']
                tmp_pool = pool.tolist()
                tmp_pool[3-order_change] = origin_amount+ticker_in
                tmp_pool[2+order_change] = counter_amount - counter_out
                reserve.iloc[candidates.iloc[i,3]] = tmp_pool
                cash_hist.append(cash)
                if ticker_in > 0:
                    tick_1 = candidates.iloc[i,2]
                    tick_2 = ticker
                    token_1 = str(counter_out)
                    token_2 = str(-ticker_in)
                else:
                    tick_2 = candidates.iloc[i,2]
                    tick_1 = ticker
                    token_2 = str(counter_out)
                    token_1 = str(-ticker_in)
                sequence_hist.append('CEX ' + tick_1 +' '+ token_1)
                sequence_hist.append('DEX '+ tick_1 + ' -> '+ tick_2 + ' ' + token_1)
                sequence_hist.append('CEX ' + tick_2 +' '+ token_2)
    return cash_hist, sequence_hist

In [12]:
# Length-2
def length_2(reserve):
    search_order = CEX_tickers
    cash = 0
    cash_hist = []
    sequence_hist = []
    for ticker in search_order:
        candidates_1 = pool_network[(pool_network['ticker']==ticker) * (pool_network['EX'] == 'DEX')]
        for i in range(candidates_1.shape[0]):
            via_asset = candidates_1.iloc[i,2]
            candidates_2 = pool_network[(pool_network['ticker']==via_asset) * (pool_network['EX'] == 'CEX') * (pool_network['counter']!=ticker)]
            for j in range(candidates_2.shape[0]):
                output_asset = candidates_2.iloc[j,2]
                pool_1 = reserve.iloc[candidates_1.iloc[i,3]]
                pool_2 = reserve.iloc[candidates_2.iloc[j,3]]
                order_change_1 = pool_1.iloc[0]==ticker
                order_change_2 = pool_2.iloc[0]==via_asset
                
                origin_amount = pool_1.iloc[3-order_change_1]
                via_amount = pool_1.iloc[2+order_change_1]
                product_1 = origin_amount*via_amount
                
                via_counter_amount = pool_2.iloc[3-order_change_2]
                counter_amount = pool_2.iloc[2+order_change_2]
                product_2 = via_counter_amount*counter_amount
                
                real_price = orderbook.loc[output_asset]['price']/orderbook.loc[ticker]['price']
                
                
                ticker_in = (-origin_amount*via_counter_amount + math.sqrt(r**2 * real_price*counter_amount*origin_amount*via_amount*via_counter_amount))/(r**2 * via_amount + r * via_counter_amount)
                if ticker_in < 0:
                    continue
                via_optimal = product_1 / (origin_amount + ticker_in)
                via_out = via_amount - via_optimal
                counter_optimal = product_2 / (via_counter_amount + via_out)
                counter_out = counter_amount - counter_optimal
                
                
                if  counter_out * orderbook.loc[output_asset]['price'] - ticker_in * orderbook.loc[ticker]['price'] > threshold:
                    cash += counter_out * orderbook.loc[output_asset]['price'] - ticker_in * orderbook.loc[ticker]['price']
                    tmp_pool_1 = pool_1.tolist()
                    tmp_pool_1[3-order_change_1] = origin_amount + ticker_in
                    tmp_pool_1[2+order_change_1] = via_optimal
                    tmp_pool_2 = pool_2.tolist()
                    tmp_pool_2[3-order_change_2] = via_counter_amount + via_out
                    tmp_pool_2[2+order_change_2] = counter_optimal
                    reserve.iloc[candidates_1.iloc[i,3]] = tmp_pool_1
                    reserve.iloc[candidates_2.iloc[j,3]] = tmp_pool_2
                    cash_hist.append(cash)
                    if ticker_in > 0:
                        tick_1 = ticker
                        tick_2 = via_asset
                        tick_3 = output_asset
                        token_1 = str(ticker_in)
                        token_2 = str(via_out)
                        token_3 = str(-counter_out)
                    else:
                        tick_1 = output_asset
                        tick_2 = via_asset
                        tick_3 = ticker
                        token_1 = str(-counter_out)
                        token_2 = str(-via_out)
                        token_3 = str(ticker_in)
                    sequence_hist.append('CEX ' + tick_1 +' '+ token_1)
                    sequence_hist.append('DEX '+ tick_1 + ' -> '+ tick_2 + ' ' + token_1)
                    sequence_hist.append('DEX '+ tick_2 + ' -> '+ tick_3 + ' ' + token_2)
                    sequence_hist.append('CEX ' + tick_3 +' '+ token_3)
    return cash_hist, sequence_hist

In [13]:
# Length-3
def length_3(reserve):
    search_order = CEX_tickers
    cash = 0
    cash_hist = []
    sequence_hist = []
    for ticker in search_order:
        candidates_1 = pool_network[(pool_network['ticker']==ticker) * (pool_network['EX'] == 'DEX')] # DEX asset을 거쳐야 하기 때문.
        for i in range(candidates_1.shape[0]):
            via_asset_1 = candidates_1.iloc[i,2]
            candidates_2 = pool_network[(pool_network['ticker']==via_asset_1) * (pool_network['EX'] == 'DEX')]
            for j in range(candidates_2.shape[0]):
                via_asset_2 = candidates_2.iloc[j,2]
                candidates_3 = pool_network[(pool_network['ticker']==via_asset_2) * (pool_network['EX'] == 'CEX')] # 처음 ticker로 가도 상관없다. 
                for k in range(candidates_3.shape[0]):
                    output_asset = candidates_3.iloc[k,2]
                    pool_1 = reserve.iloc[candidates_1.iloc[i,3]]
                    pool_2 = reserve.iloc[candidates_2.iloc[j,3]]
                    pool_3 = reserve.iloc[candidates_3.iloc[k,3]]
                    order_change_1 = pool_1.iloc[0]==ticker
                    order_change_2 = pool_2.iloc[0]==via_asset_1
                    order_change_3 = pool_3.iloc[0]==via_asset_2

                    origin_amount = pool_1.iloc[3-order_change_1]
                    via_amount = pool_1.iloc[2+order_change_1]
                    product_1 = origin_amount*via_amount # Caching is better option.

                    via_1_amount = pool_2.iloc[3-order_change_2]
                    via_2_amount = pool_2.iloc[2+order_change_2]
                    product_2 = via_1_amount*via_2_amount
                    
                    via_counter_amount = pool_3.iloc[3-order_change_3]
                    counter_amount = pool_3.iloc[2+order_change_3]
                    product_3 = via_counter_amount*counter_amount

                    real_price = orderbook.loc[output_asset]['price']/orderbook.loc[ticker]['price']

                    var_1 = -origin_amount*via_1_amount*via_counter_amount
                    var_2 = math.sqrt(r**3 * real_price*product_1*product_2*product_3)
                    var_3 = r**3*via_amount*via_2_amount + r**2*via_amount*via_counter_amount + r*via_1_amount*via_counter_amount

                    ticker_in = -(-var_1 - var_2)/(var_3)
                    if ticker_in < 0:
                        continue
                    via_1_optimal = product_1 / (origin_amount + ticker_in)
                    via_1_out = via_amount - via_1_optimal
                    via_2_optimal = product_2 / (via_1_amount + via_1_out)
                    via_2_out = via_2_amount - via_2_optimal
                    counter_optimal = product_3 / (via_counter_amount + via_2_out)
                    counter_out = counter_amount - counter_optimal

                    if  counter_out * orderbook.loc[output_asset]['price'] - ticker_in * orderbook.loc[ticker]['price'] > threshold:
                        cash += counter_out * orderbook.loc[output_asset]['price'] - ticker_in * orderbook.loc[ticker]['price']
                        tmp_pool_1 = pool_1.tolist()
                        tmp_pool_1[3-order_change_1] = origin_amount + ticker_in
                        tmp_pool_1[2+order_change_1] = via_1_optimal
                        
                        tmp_pool_2 = pool_2.tolist()
                        tmp_pool_2[3-order_change_2] = via_1_amount + via_1_out
                        tmp_pool_2[2+order_change_2] = via_2_optimal
                        
                        tmp_pool_3 = pool_3.tolist()
                        tmp_pool_3[3-order_change_3] = via_counter_amount + via_2_out
                        tmp_pool_3[2+order_change_3] = counter_optimal
                        reserve.iloc[candidates_1.iloc[i,3]] = tmp_pool_1
                        reserve.iloc[candidates_2.iloc[j,3]] = tmp_pool_2
                        reserve.iloc[candidates_3.iloc[k,3]] = tmp_pool_3
                        cash_hist.append(cash)
                        if ticker_in > 0:
                            tick_1 = ticker
                            tick_2 = via_asset_1
                            tick_3 = via_asset_2
                            tick_4 = output_asset
                            token_1 = str(ticker_in)
                            token_2 = str(via_1_out)
                            token_3 = str(via_2_out)
                            token_4 = str(-counter_out)
                        else:
                            tick_1 = output_asset
                            tick_2 = via_asset_2
                            tick_3 = via_asset_1
                            tick_4 = ticker
                            token_1 = str(-counter_out)
                            token_2 = str(-via_2_out)
                            token_3 = str(-via_1_out)
                            token_4 = str(ticker_in)
                        sequence_hist.append('CEX ' + tick_1 +' '+ token_1)
                        sequence_hist.append('DEX '+ tick_1 + ' -> '+ tick_2 + ' ' + token_1)
                        sequence_hist.append('DEX '+ tick_2 + ' -> '+ tick_3 + ' ' + token_2)
                        sequence_hist.append('DEX '+ tick_3 + ' -> '+ tick_4 + ' ' + token_3)
                        sequence_hist.append('CEX ' + tick_4 +' '+ token_4)
    return cash_hist, sequence_hist

In [14]:
a, b = length_1(reserves) # nothing

a2_hist, b2_hist = [], []
a3_hist, b3_hist = [], []
terminal = False
counter = 0
for i in range(2):
    counter+=1
    a2, b2 = length_2(reserves)
    a3, b3 = length_3(reserves)
    if len(a2) > 0:
        a2_hist.append(a2)
        b2_hist.append(b2)
    if len(a3) > 0:
        a3_hist.append(a3)
        b3_hist.append(b3)
    if len(a2)==0 and len(a3)==0:
        terminal = True